In [15]:
!pip install -q torch torchvision h5py huggingface_hub tqdm


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [16]:
# Thư viện
import os, sys, h5py, numpy as np, torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
from torch.cuda.amp import GradScaler, autocast
from tqdm.auto import tqdm   # tqdm.auto hiển thị tốt trên Kaggle
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
# ===== 1. Đường dẫn =====
INPUT_DIR = "/kaggle/input/galaxy10-decals"         # thay bằng tên dataset của bạn
H5_PATH   = f"{INPUT_DIR}/Galaxy10_DECals.h5"
SAVE_PATH = "/kaggle/working/galaxy10_b0.pth"       # checkpoint & đồ thị sẽ lưu ở đây
assert os.path.exists(H5_PATH), f"❌ Không tìm thấy file HDF5: {H5_PATH}"
print("✅ Đã tìm thấy:", H5_PATH)

✅ Đã tìm thấy: /kaggle/input/galaxy10-decals/Galaxy10_DECals.h5


In [19]:
# Tham số, augmentation
IMG_SIZE = 128
MEAN, STD = [0.5]*3, [0.5]*3
AUGMENT = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((IMG_SIZE, IMG_SIZE), antialias=True),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(180),
    transforms.ToTensor(),
    transforms.Normalize(MEAN, STD),
])

In [20]:
# Dataset 
class Galaxy10H5(Dataset):
    def __init__(self, h5_path, indices=None, transform=None):
        self.h5   = h5py.File(h5_path, "r")
        self.imgs = self.h5["images"]      # (N,256,256,3)
        self.labs = self.h5["ans"]         # (N,)
        self.idxs = np.arange(len(self.imgs)) if indices is None else indices
        self.tfm  = transform
    def __len__(self):  return len(self.idxs)
    def __getitem__(self, i):
        img = self.imgs[self.idxs[i]]      # uint8 RGB
        lab = int(self.labs[self.idxs[i]])
        if self.tfm: img = self.tfm(img)
        return img, lab

In [21]:
# Khởi tạo tập & DataLoader
full_ds = Galaxy10H5(H5_PATH, transform=AUGMENT)
N       = len(full_ds)
n_val   = int(0.10 * N)
n_test  = int(0.10 * N)
n_train = N - n_val - n_test
g       = torch.Generator().manual_seed(42)
train_ds, val_ds, test_ds = random_split(full_ds, [n_train, n_val, n_test], generator=g)
BATCH = 64
NUM_WORKERS = min(8, os.cpu_count())      # Kaggle thường 4-8 logic cores
train_loader = DataLoader(train_ds, batch_size=BATCH, shuffle=True,
                          num_workers=NUM_WORKERS, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=BATCH, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=True)
print(f"Train {len(train_ds)} | Val {len(val_ds)} | Test {len(test_ds)}")

Train 14190 | Val 1773 | Test 1773


In [22]:
# Thiết lập thiết bị & mô hình
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)
NUM_CLASSES = 10
model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, NUM_CLASSES)
model = model.to(device)
criterion  = torch.nn.CrossEntropyLoss(label_smoothing=0.05)
optimizer  = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
scheduler  = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)
scaler     = GradScaler()                            # Mixed Precision

Using: cpu


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth

  0%|          | 0.00/20.5M [00:00<?, ?B/s]
100%|██████████| 20.5M/20.5M [00:00<00:00, 123MB/s] 
/tmp/ipykernel_10/3520420023.py:11: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler     = GradScaler()                            # Mixed Precision


In [ ]:
# Vòng lặp huấn luyện 
EPOCHS      = 10
PATIENCE    = 5
best_val_acc = 0
patience_cnt = 0
history = {"train_loss":[], "val_loss":[], "train_acc":[], "val_acc":[]}
def run_epoch(loader, train=True):
    if train:  model.train()
    else:      model.eval()
    total_loss = correct = total = 0
    torch.set_grad_enabled(train)
    loop = tqdm(loader, leave=False)
    for x, y in loop:
        x, y = x.to(device), y.to(device)
        if train:
            optimizer.zero_grad()
            with autocast():
                logits = model(x)
                loss   = criterion(logits, y)
            scaler.scale(loss).backward()
            scaler.step(optimizer); scaler.update()
        else:
            with torch.no_grad(), autocast():
                logits = model(x)
                loss   = criterion(logits, y)
        total_loss += loss.item() * x.size(0)
        preds = logits.argmax(1)
        correct += (preds == y).sum().item()
        total   += x.size(0)
        loop.set_postfix(loss=loss.item())
    avg_loss = total_loss / total
    avg_acc  = correct / total
    return avg_loss, avg_acc
for epoch in range(1, EPOCHS+1):
    tr_loss, tr_acc = run_epoch(train_loader, train=True)
    val_loss, val_acc = run_epoch(val_loader, train=False)
    scheduler.step()
    history["train_loss"].append(tr_loss); history["val_loss"].append(val_loss)
    history["train_acc"].append(tr_acc);   history["val_acc"].append(val_acc)
    print(f"Epoch {epoch:02d} | "
          f"train_loss={tr_loss:.3f} val_loss={val_loss:.3f} "
          f"train_acc={tr_acc:.3%} val_acc={val_acc:.3%}")
    # Early-stopping + save best
    if val_acc > best_val_acc:
        best_val_acc = val_acc; patience_cnt = 0
        torch.save({"model": model.state_dict(),
                    "acc": best_val_acc,
                    "epoch": epoch,
                    "img_size": IMG_SIZE}, SAVE_PATH)
        print("  ✅ Saved new best model")
    else:
        patience_cnt += 1
        if patience_cnt >= PATIENCE:
            print("  ⏹ Early-stopping triggered")
            break


  0%|          | 0/222 [00:00<?, ?it/s]/tmp/ipykernel_10/3184695838.py:17: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():

100%|██████████| 222/222 [20:39<00:00,  3.31s/it, loss=1.04]
                                                            
  0%|          | 0/28 [00:00<?, ?it/s]/tmp/ipykernel_10/3184695838.py:23: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), autocast():

100%|██████████| 28/28 [02:58<00:00,  5.13s/it, loss=0.897]
                                                           

Epoch 01 | train_loss=1.398 val_loss=1.011 train_acc=55.285% val_acc=72.081%
  ✅ Saved new best model



100%|██████████| 222/222 [20:44<00:00,  3.20s/it, loss=0.85]
                                                            
 86%|████████▌ | 24/28 [02:14<00:05,  1.27s/it, loss=0.8]

In [ ]:
# Đánh giá trên tập test
ckpt = torch.load(SAVE_PATH, map_location=device)
model.load_state_dict(ckpt["model"]); model.eval()
all_preds, all_labels = [], []
with torch.no_grad(), autocast():
    for x, y in tqdm(test_loader, leave=False):
        logits = model(x.to(device))
        preds  = logits.argmax(1).cpu()
        all_preds.append(preds); all_labels.append(y)
all_preds  = torch.cat(all_preds).numpy()
all_labels = torch.cat(all_labels).numpy()
print("Test accuracy:", (all_preds == all_labels).mean()*100, "%")
print(classification_report(all_labels, all_preds,
      target_names=[f"class_{i}" for i in range(NUM_CLASSES)]))

In [ ]:
# Vẽ Learning Curve
plt.figure(figsize=(5,4))
plt.plot(history["train_loss"], label="train_loss")
plt.plot(history["val_loss"],   label="val_loss")
plt.xlabel("epoch"); plt.ylabel("loss"); plt.legend(); plt.show()
plt.figure(figsize=(5,4))
plt.plot(history["train_acc"], label="train_acc")
plt.plot(history["val_acc"],   label="val_acc")
plt.xlabel("epoch"); plt.ylabel("accuracy"); plt.legend(); plt.show()